In [ ]:
import util_xtda_chem, pickle,torch, json,cv2
import cohortintgrad as csig
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
util_xtda_chem.map_js_dump(df_fn='tobacco_1.0_CH4_100bar_sc.csv',
                           output_fn='map_from_testID_to_cifID_tobacco.json', 
                           check_npz_fn='tobacco_1.0_CH4_100bar_x_sc.npy')

In [ ]:
cif_id = 4921
with open("map_from_testID_to_cifID_tobacco.json", mode="r") as f:
    map_js = json.load(f)
data_id=np.where(np.array(list(map_js.values())) == cif_id)[0].item()

In [ ]:
_, x, _, _ = util_xtda_chem.dataload(npz_fn='tobacco_1.0_CH4_100bar_x_sc.npy', df_fn='tobacco_1.0_CH4_100bar_sc.csv')
num_data = 1000
x = x[:num_data]

model_fn = 'model_tobacco.pkl'
with open(model_fn, mode="rb") as f:
    rf = pickle.load(f)

y = rf.predict(x.reshape(x.shape[0], -1))

In [ ]:
IG = csig.CohortIntGrad(torch.Tensor(x), torch.Tensor(y), ratio=0.01, n_step=50)
ig = IG.igcs_single(t_id=data_id)
torch.save(ig, f'igcs_tobacco_100bar_cube_predict_single{cif_id}')

In [ ]:
print(y[data_id])

In [ ]:
util_xtda_chem.plot_original_csig_fromsingle(
    data_id=np.where(np.array(list(map_js.values())) == cif_id)[0].item(),
    cif_id=cif_id,
    x=x,
    igcs_fn = f'igcs_tobacco_100bar_cube_predict_single{cif_id}',
    dgm_dir = "tobacco_1.0_CH4_100bar/",
    px_size = 54,
    onemaxB = 27.0,
    twomaxB = 27.0,
    onemaxP = 8.8,
    twomaxP = 3.5,
    db_prefix = "tobmof-",
    prefix="igcs_tobacco_plot"
)

In [ ]:
plt.imshow(cv2.cvtColor(cv2.imread(f"igcs_tobacco_plot_{cif_id}.png"),cv2.COLOR_BGR2RGB))
plt.show()